In [77]:
import pandas as pd
import json
import time
from multiprocessing import Pool, cpu_count
from analyse import *

import inspect
# 获取类的所有方法名并执行
def execute_all_methods(cls,instance=None):
    # 如果没有传入实例，则创建一个实例
    if instance is None:
        instance = cls()

    # 获取类中所有方法（排除私有方法和特殊方法）
    methods = [
        method_name for method_name, method_obj in inspect.getmembers(cls, predicate=inspect.isfunction)
        if not method_name.startswith("_")  # 排除私有方法和特殊方法
    ]
     
    ignore_name =['Aroon_2', 'BBANDS_3', 'HT_PHASOR', 'HT_SINE', 'MACDFIX_3', 
                  'MACD_3', 'MININDEX', 'STOCH', 'STOCHF', 'STOCHRSI', 'STOCH_2']
    special_name =['Aroon_2', 'BBANDS_3', 'HT_PHASOR', 'HT_SINE', 'MACDFIX_3', 
                  'MACD_3', 'MININDEX', 'STOCH', 'STOCHF', 'STOCHRSI', 'STOCH_2']
    res ={}
    for method_name in methods:
        method = getattr(instance, method_name)
        vals = method()
        # if method_name in ignore_name:
        #     continue
        if method_name in special_name:
            if isinstance(vals,dict):
                for k,val in vals.items():
                    subname = k
                    res[subname] = val.item()
            else:
                 for i,val in enumerate(vals):
                    subname = method_name+f"_{i+1}"
                    res[subname] = val.item()
        else:
            res[method_name] = vals.item()
        
    return res






file_path = 'datas/BTC_USDT-1s.feather'

data = pd.read_feather(file_path)


In [78]:
len(data)

129554417

In [110]:
import random
low = 0
high = len(data)-5000
count = 200000
import os
if not os.path.exists("/data/users/zzp/Trade/datas/indexs"):
    random.seed(18)
    numbers = random.sample(list(range(high)),count)
    with open("/data/users/zzp/Trade/datas/indexs",'w')as f:
        f.write(json.dumps(numbers))
with open("/data/users/zzp/Trade/datas/indexs",'r')as f:
        numbers = json.loads(f.read())

In [111]:
all_data = [data.iloc[index:index+3600+300] for index in numbers]
import pandas as pd


In [112]:
len(all_data)

200000

In [113]:
all_data[0]

,date,open,high,low,close,volume
24328958,2021-12-21 03:44:15+00:00,47421.96,47421.96,47416.43,47416.43,0.92752
24328959,2021-12-21 03:44:16+00:00,47415.38,47415.38,47408.03,47408.04,0.62228
24328960,2021-12-21 03:44:17+00:00,47408.04,47408.04,47408.03,47408.03,0.10387
24328961,2021-12-21 03:44:18+00:00,47408.03,47408.03,47408.03,47408.03,0.01829
24328962,2021-12-21 03:44:19+00:00,47408.04,47408.04,47408.03,47408.03,0.40759
...,...,...,...,...,...,...
24332853,2021-12-21 04:49:10+00:00,48561.97,48561.97,48549.08,48550.00,2.37263
24332854,2021-12-21 04:49:11+00:00,48550.00,48559.89,48550.00,48552.38,0.14993
24332855,2021-12-21 04:49:12+00:00,48555.16,48555.17,48555.16,48555.17,0.12799
24332856,2021-12-21 04:49:13+00:00,48555.16,48555.16,48549.09,48549.10,0.60485


In [101]:
 
def reorganize_stock_data(df,tp="s",is_smooth=True):
    if tp == "s":
        k = 15
    elif tp =='d':
        k=3600*24
    elif tp == 'm':
        k = 60
    elif tp =='h':
        k = 3600
    # 确保数据按时间顺序排列（假设索引是时间顺序）
    df = df.sort_index()  # 如果索引不是时间戳，可能需要按时间列排序
    # 创建分组键（每15秒为一组）
    df['group'] = df.index // k  # 基于索引分组（假设索引从0开始连续）
    
    # 定义聚合规则（假设价格列名为'price'，成交量为'volume'）
    agg_dict = {
        'open':   'first',  # 每组第一个价格作为开盘价
        'high':  'max',    # 每组最高价
        'low':   'min',    # 每组最低价
        'close':  'last',   # 每组最后一个价格作为收盘价
        'volume': 'sum'    # 每组成交量总和
    }
    # 执行聚合操作
    ohlcv_df = df.groupby('group').agg(agg_dict)
    # 重置索引并清理列名（可选）
    ohlcv_df = ohlcv_df.reset_index(drop=True)
    if is_smooth:
        window = 5  # 5日平滑
        cols_to_smooth = ['open', 'high', 'low', 'close']
        ohlcv_df[cols_to_smooth] = ohlcv_df[cols_to_smooth].rolling(window=window).mean()
    return ohlcv_df



In [84]:
import matplotlib.pyplot as plt
def plot(data):
    # 创建 x 轴数据（索引）
    x = np.arange(len(data))
    plt.plot(x, data, marker='o', linestyle='-', color='b')
    plt.xlabel("Index")
    plt.ylabel("Value")
    plt.grid(True)
    plt.legend()
    # 显示图形
    plt.show()

In [102]:
reorganize_stock_data(all_data[0],'m',False)

,open,high,low,close,volume
0,47421.96,47421.96,47380.88,47385.00,6.61743
1,47385.00,47405.99,47361.92,47403.68,16.85909
2,47401.54,47470.03,47399.15,47460.09,13.33319
3,47460.10,47490.36,47453.09,47490.35,12.78167
4,47490.36,47499.62,47477.80,47485.59,7.13766
...,...,...,...,...,...
61,48614.92,48630.00,48599.76,48629.99,22.37982
62,48630.00,48665.73,48627.39,48662.80,24.37346
63,48662.80,48663.57,48622.24,48626.11,27.24718
64,48623.01,48629.00,48579.79,48601.01,33.40522


In [103]:

def smooth(data,window=5):
    a,b = data[0],data[-1]
    weights = np.repeat(1.0, window)/window
     # 创建权重数组
    sma = np.convolve(data, weights, 'valid')  # 使用卷积计算移动平均
    sma = np.concatenate((
        [a],
        sma,
        [b]
        )
    )
    return sma 
def is_steady(data):
    return data.max()-data.min()<0.03

def is_up(data):
    diff = np.diff(data)
    return (diff>0).sum()>0.7*len(data)
def is_down(data):
    diff = np.diff(data)
    return (diff<0).sum()>0.7*len(data)
def is_u_shape(data):
    diff = np.diff(data)
    diff = diff.sum()/len(diff)
    max_index = np.argmax(data)
    flag = data[:3].sum()- data[-3:].sum()
    flag = abs(flag)
    flag = flag<9*diff
    if is_up(data[:max_index]) and is_down(data[max_index:]):
        return True
    return False
def is_r_u_shape(data):
    return is_u_shape(-data)
def merge_trend(trend):
    res = []
    for item in trend:
        if len(res)==0 or res[-1]!= str(int(item[0])):
            res.append(str(int(item[0])))
    return "".join(res)
def get_trend(data):
    res = []
    for i in range(1,len(data)):
        trend = data[i]>=data[i-1]
        if len(res)==0:
            res.append([trend,0,data[i]])
            continue
        if res[-1][0]==trend:
            res[-1][2] = data[i]
        else:
            res.append([trend,res[-1][2],data[i]])
    return merge_trend(res)
def finalcheck(data):
    pattern_id = get_trend(data)
    return pattern_id
    # # print("patten ", pattern_id)
    # if pattern_id in ["00","1","0","10","01","010",'101',"0101","1010","10101","01010"]:
    #     return pattern_id
    # return "-1"


def checkpattern(data):
    if is_steady(data):
        return "--"
    # elif is_down(data):
    #     return "0"
    # elif is_up(data):
    #     return "1"
    # elif is_u_shape(data):
    #     return "01"
    # elif is_r_u_shape(data):
    #     return "10"
    else:
        return finalcheck(data)

In [114]:
def get_id(data):
    testdata = reorganize_stock_data(data,'m',False)
    testdata = testdata['close'].values
    start = testdata[0]
    testdata = (testdata-start)/start
    # print(testdata)
    # plot(testdata)
    testdata = smooth(testdata,5)
    testdata = smooth(testdata,5)
    testdata = smooth(testdata,2)
    # testdata = smooth(testdata,15)
    # plot(testdata)
    patternid = checkpattern(testdata)
    return patternid
# testdata


In [ ]:
# mp=[]
# for i in range(1000):
#     testdata = reorganize_stock_data(all_data[i],'m',False)
#     testdata = testdata['close'].values
#     start = testdata[0]
#     testdata = (testdata-start)/start
#     v = f"{testdata[-1]-testdata[-6]:.3f}"
#     mp.append(v)



In [108]:
import multiprocessing
import numpy as np
import re
def process_single_data(i):
    testdata = reorganize_stock_data(all_data[i], 'm', False)
    testdata = testdata['close'].values
    start = testdata[0]
    testdata = (testdata - start) / start
    v = f"{testdata[-1]-testdata[-6]:.3f}"
    return eval(v)
def run_multiprocessing():
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count()-1)  # 使用比CPU核心数少一个的进程数量
    results = pool.map(process_single_data, range(200000))
    pool.close()
    pool.join()
    return results
mp = run_multiprocessing()
print(len(mp))

200000


In [ ]:
# mp = {}
# for i in range(len(all_data)//100):
#     p = get_id(all_data[i])
#     if p not in mp:
#         mp[p]=0
#     mp[p]+=1
# mp

{'--': 1986,
 '01010101': 2,
 '01010': 1,
 '1010': 1,
 '1010101010': 1,
 '10101010': 2,
 '101010': 2,
 '0101010': 1,
 '010': 1,
 '0101010101': 1,
 '010101': 1,
 '010101010': 1}

In [115]:
mp.sort()
mp[len(mp)//2]

0.0

In [116]:
mp1={}
for v in mp:
    if v not in mp1:
        mp1[v]=0
    mp1[v]+=1


In [117]:
cnt1=0
cnt2=0
v1 = -0.004
v2 = 0.004
for k in mp:
    if k<=v1:
        cnt1+=1
    if k>=v2:
        cnt2+=1
print(cnt1,cnt2)

4763 4644


In [118]:
len(all_data[0]['close'].values)

3900

In [122]:
# cls1,cls2,cls3
training_mp = {0:[],1:[],-1:[]}
for i in range(len(all_data)): 
    curdata = all_data[i]['close'].values[-301:]
    v = curdata[0]
    curdata = (curdata-v)/v
    curv = curdata[-1]
    if curv<=v1:
        training_mp[-1].append(i)
    elif curv>=v2:
        training_mp[1].append(i)
    else:
        training_mp[0].append(i)


In [123]:
with open("datas/class.json",'w')as f:
    f.write(json.dumps(training_mp))